In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/crypto_model/model.h5" 'model.h5'
!cp "/content/drive/My Drive/crypto_model/util.py" 'util.py'
!cp "/content/drive/My Drive/crypto_model/btcData.csv" 'btcData.csv'

Mounted at /content/drive


In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

In [ ]:
#@title Main Functions
!pip install pandas_ta==0.3.14b
!pip install python_binance   ta 


import sys
import os
#sys.path.append(os.path.abspath("/content"))
#from util1 import *
from tensorflow.keras.models import load_model
import time
import pandas as pd
from sklearn.preprocessing import scale as scaler
import numpy as np
import pandas as pd
import pandas_ta as pta
from ta import add_all_ta_features
import requests
import numpy as np
from datetime import datetime, timedelta
from binance.client import Client
from binance.enums import HistoricalKlinesType
from util import *


columns = columns[0].split("+")
model = load_model('model.h5')

model = load_model('model.h5')
#load_model(filepath)

class Get_data:
  def __init__(self , client ,interval, coin = "XRPUSDT", start = "1 Jan, 2021", end = "12 May, 2021"):
    self.client = client
    self.interval = interval
    self.coin = coin
    self.start = start
    self.end = end
    #self.df = get_data(clint,interval, coin , start , end )
  def get_data(self):
      columns = [ "Open time", "open","high", " low"  ,"close", "volume" , 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
      #columns = ["Open time", "open","high", " low"  ,"close", "volume"]
      candlesticks = self.client.get_historical_klines(self.coin, self.interval,self.start , self.end, klines_type  = HistoricalKlinesType.SPOT  )
      df = pd.DataFrame(candlesticks, columns = columns)
      #columns = [ "Open time", "open","high", " low"  ,"close", "volume" , 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
      #df = df[columns]
      df["Open time"] = pd.to_datetime(df["Open time"] ,unit='ms')
      return df
#Get_data(client,Client.KLINE_INTERVAL_30MINUTE, "XRPUSDT", "1 Jan, 2021", "12 May, 2021").get_data()



class AddIndictors:
  def __init__(self, df):
    self.df = df 
  #Encoding datetime
  def add_time(self , df1):
    time  = df1['timestamp']
    minute = time.apply(lambda x : x.minute)
    hr = time.apply(lambda x : x.hour)
    day = time.apply(lambda x : x.day)
    weekday = time.apply(lambda x : x.weekday())
    mnth = time.apply(lambda x : x.month)

    df1['hr_sin'] = np.sin(hr*(np.pi/24))
    df1['hr_cos'] = np.cos(hr*(np.pi/24))
    df1['day_sin'] = np.sin((day-1)*(2.*np.pi/30))
    df1['day_cos'] = np.cos((day-1)*(2.*np.pi/30))
    df1['wkdy_sin'] = np.sin(weekday*(2.*np.pi/7))
    df1['wkdy_cos'] = np.cos(weekday*(2.*np.pi/7))
    df1['mnth_sin'] = np.sin((mnth-1)*(2.*np.pi/12))
    df1['mnth_cos'] = np.cos((mnth-1)*(2.*np.pi/12))

    df= df1
    df.set_index("timestamp", inplace=True)
    df.ta.strategy('All')
    nan_columns = []
    for column in df.columns:
      if df[column].isnull().sum() > 300:
        nan_columns.append(column)
    df[nan_columns] =df[nan_columns].fillna(0)
    df.reset_index(drop=True, inplace=True)
    return df


  def add_indictors(self,df , moving_a = [7, 9,14,20, 25,50,100, 300], rsi_r = [2,3, 4, 6,10,14,20, 30,100]):
    df[['open', 'high', 'low', 'close', 'volume']] =df[['open', 'high', 'low', 'close', 'volume']].astype('float')
    for range in moving_a:
      df.ta.sma(length=range, append = True)
      df.ta.ema(length=range, append = True)
    for range in rsi_r:
      df.ta.rsi(length=range, append = True)
    df = df.assign(hig_close=df['high'].astype(float) - df['close'].astype(float))
    df = df.assign(low_close=df['low'].astype(float) - df['close'].astype(float))
    df = df.assign(EMA_20_EMA_50=df['EMA_20'].astype(float) - df['EMA_50'].astype(float)) # EMA20 - EMA50 

    #EMA
    df = df.assign(EMA_9_close=df['EMA_7'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(EMA_9_close=df['EMA_9'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(EMA_25_close=df['EMA_25'].astype(float) - df['close'].astype(float)) # EMA25 - close 
    df = df.assign(EMA_100_close=df['EMA_100'].astype(float) - df['close'].astype(float)) # EMA100 - clpse 
    df = df.assign(EMA_20_EMA_50=df['EMA_20'].astype(float) - df['EMA_50'].astype(float)) # EMA20 - EMA50
    df = df.assign(EMA_9_EMA_29=df['EMA_9'].astype(float) - df['EMA_9'].astype(float)) # EMA20 - EMA50

    #SMA
    df = df.assign(SMA_9_close=df['SMA_7'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(SMA_9_close=df['SMA_9'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(SMA_25_close=df['SMA_25'].astype(float) - df['close'].astype(float)) # EMA25 - close 
    df = df.assign(SMA_100_close=df['SMA_100'].astype(float) - df['close'].astype(float)) # EMA100 - clpse 
    df = df.assign(SMA_20_EMA_50=df['SMA_20'].astype(float) - df['SMA_50'].astype(float)) # EMA20 - EMA50
    df = df.assign(SMA_9_EMA_29=df['SMA_9'].astype(float) - df['SMA_9'].astype(float)) # EMA20 - EMA50

    try:
      df = df.assign(Taker_seller_base_asset_volume=df['volume'].astype(float) - df['Taker buy base asset volume'].astype(float))
      df = df.assign(Taker_seller_quote_asset_volume=df['Quote asset volume'].astype(float) - df['Taker buy quote asset volume'].astype(float))
    except:pass

    return df

  def get_indictors(self):
    self.df == self.add_time(self.df)
    print("adding indictors...", len(self.df.columns))
    df = add_all_ta_features(self.df,  open="open", high="high", low="low", close="close", volume="volume", fillna=True)
    print("adding ...", len(df.columns))
    df = self.add_indictors(df)
    return df




def prepare_data(df):
  data = df[columns]
  data = data.astype(float)
  values_after_scale = scaler(data)
  data= pd.DataFrame(values_after_scale,columns=data.columns)
  return data

#data = prepare_data(df)

In [ ]:
#@title Getting data from Binance
api_key = "zRbnRTq5K7R8TFoF2W0wcfx4Jz9al3MYTrzxHbbpWjPZhiq7fLzsKUpcEc63jHyG"
secret_key = "XVqunTEKoLMRz9FGuXbo1ZXbLnhTdQnRs0St3d5CrHz59kC9XvFV3E16L2f9kQWC"
client = Client(api_key, secret_key, requests_params = {"timeout":100 } )
print("Getting data ...")
df1 = Get_data(client,interval = Client.KLINE_INTERVAL_15MINUTE, coin = "BTCUSDT", start = "16 Aug, 2022", end = "16 Dec, 2025").get_data()#[-300:]
df1.head(2)

In [ ]:
#@title Getting data from local file
df_local = pd.read_csv("btcData.csv")
del df_local['Unnamed: 0']
df_local.columns = [x.lower() for x in df_local.columns]
df_local['low'] = df_local[' low']
del df_local[' low']
y= df_local['action']
x = df_local.drop('action', axis = 1)
df_local.head(3)

In [ ]:
df['profit'].sum(), 
#for x in range(len(df['profit'])):
#  print(df['action'][x] , df['profit'][x])



(1450536.6699999997,)

In [ ]:
df = data = df_orignal = df_local
df['profit'] = np.zeros(len(df))
already_buyed = False
buy_at = 0
profit = 0
x_train=data
holding =0
funds = 100

for i in range(len(data)):
    label = data['action'][i]
    if label == 1 and funds > 0:
      #print("buying", bp)
      buy_at = df.open[i]
      holding += (funds)/df.open[i]
      already_buyed = True
    if label == 0 and  already_buyed == True:
      #selling
      profit = df.close[i] - buy_at
      df.profit[i] = profit
      already_buyed = False
def fitness(labels):
  df_orignal["Action"] = labels
  df = df_orignal
  holding = 0
  funds = initial_amount =100
  #net =model #neat.nn.FeedForwardNetwork.create(genome, config)
  hasSold = False
  tax = 0.00085
  #hasBought = False
  playAmount = 3000
  Transaction = 0
  for i in range(len(labels)):
    signal =df.Action[i] #np.argmax(model.predict(np.array([x_test[i]])))  #Label code : BUY => 1, SELL => 0, HOLD => 2
    #print(signal)
    if signal == 2: # Hold
      #print("Holding ..")
      continue
    if signal == 1 and funds > 0: # Buy, price going up, or going long
      print(funds)
      holding += (funds)/df.open[i] #Total amoutn invested - text  then divide #(Open)
      funds = 0
      hasBought = True
      Transaction =Transaction + 1
      #print("Buying ...")
    if signal == 0 and holding > 0: # selling #price going down, going short
      print(holding)
      funds = holding * df.close[i] 
      holding = 0
      hasSold = True
      Transaction  =Transaction+ 1
      #print("Selling ...")

    score = funds + holding*df.close[i]
    #print(f"Transaction {Transaction}, Funds: {funds}, Hold ETH: {holding}, Current score: {score}, Current Score in ETH:{score/df.close[i]}")
  return score


In [ ]:
fitness(df['action'])

Streaming output truncated to the last 5000 lines.
9.069517694479697e+32
2.3333544369871257e+28
9.087955861240768e+32
2.3447900053977823e+28
9.228490850214284e+32
2.344876394950471e+28
9.215139124021436e+32
2.344876394950471e+28
9.26083607520623e+32
2.365580507334484e+28
9.17441195695127e+32
2.3680656289059505e+28
9.21177056031289e+32
2.3680656289059505e+28
9.237329092645673e+32
2.442105262605045e+28
9.218888755807743e+32
2.442729052214922e+28
9.226666405109994e+32
2.443808231619523e+28
9.309191365283554e+32
2.470886622446134e+28
9.302734938539102e+32
2.4708859661575133e+28
9.371277315240312e+32
2.4723122453635455e+28
9.382907072042501e+32
2.4984215485847026e+28
9.471091359021348e+32
2.514758593475006e+28
9.515077001579821e+32
2.514758593475006e+28
9.5113073784482e+32
2.5408259082800307e+28
9.453874549617439e+32
2.5930750618290767e+28
9.488515439368412e+32
2.656368229503686e+28
9.422642820013181e+32
2.656256653230261e+28
9.48503828879756e+32
2.6719945644208836e+28
9.525663294155015e+32

6.66419926238233e+36

In [ ]:
int(6.664199262382343e+38)

666419926238234323128083164164987551744

In [ ]:
#@title Features importances functions { display-mode: "form" }

from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.ensemble import GradientBoostingRegressor


def features_importance(x,y):
  forest = RandomForestClassifier(random_state=0)
  forest.fit(x, y)

  importances = forest.feature_importances_
  std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

  forest_importances = pd.Series(importances, index=x.columns)
  fig, ax = plt.subplots()
  forest_importances.plot.bar(yerr=std, ax=ax)
  ax.set_title("Feature importances using MDI")
  ax.set_ylabel("Mean decrease in impurity")
  fig.tight_layout()
  return forest


def features_importance_gb(x,y):
  # forest = RandomForestClassifier(random_state=0)
  # forest.fit(x, y)
  gb = GradientBoostingRegressor(n_estimators=100)
  gb.fit(x, y)
  forest = gb

  plt.rcParams["figure.figsize"] = (15.5,5.5)
  # plt.bar(range(x.shape[1]), gb.feature_importances_)
  # plt.xticks(range(x.shape[1]),x.columns )

  importances = forest.feature_importances_
  #std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

  forest_importances = pd.Series(importances, index=x.columns)
  fig, ax = plt.subplots()
  forest_importances.plot.bar( ax=ax)
  ax.set_title("Feature importances using MDI")
  ax.set_ylabel("Mean decrease in impurity")
  fig.tight_layout()
  return forest





def per_importance(x,y, forest):
  result = permutation_importance(
      forest, x, y, n_repeats=10, random_state=42, n_jobs=2
  )

  forest_importances = pd.Series(result.importances_mean, index= x.columns)

  fig, ax = plt.subplots()
  forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
  ax.set_title("Feature importances using permutation on full model")
  ax.set_ylabel("Mean accuracy decrease")
  fig.tight_layout()
  plt.show()

def get_columns(gb, limit):
  columns = []
  for val in gb.feature_importances_:
    if val > limit:
      columns.append(x.columns[list(gb.feature_importances_).index(val)])
  return columns

# ANOVA feature selection for numeric input and categorical output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
def anova_feature():
  # define feature selection
  fs = SelectKBest(score_func=f_classif, k=2)
  # apply feature selection
  X_selected = fs.fit_transform(x, y)
  print(X_selected.shape)

  forest_importances = pd.Series(fs.scores_, index=x.columns)
  fig, ax = plt.subplots()
  forest_importances.plot.bar( ax=ax)
  ax.set_title("Feature importances using MDI")
  ax.set_ylabel("Mean decrease in impurity")


In [ ]:
#@title Adding Indictors
import numpy as np
from datetime import datetime, timedelta


class AddIndictors_test:
  def __init__(self, df):
    self.df = df 
  #Encoding datetime
  def add_time(self , df1):
    time  = df1['timestamp']
    minute = time.apply(lambda x : x.minute)
    hr = time.apply(lambda x : x.hour)
    day = time.apply(lambda x : x.day)
    weekday = time.apply(lambda x : x.weekday())
    mnth = time.apply(lambda x : x.month)

    df1['hr_sin'] = np.sin(hr*(np.pi/24))
    df1['hr_cos'] = np.cos(hr*(np.pi/24))
    df1['day_sin'] = np.sin((day-1)*(2.*np.pi/30))
    df1['day_cos'] = np.cos((day-1)*(2.*np.pi/30))
    df1['wkdy_sin'] = np.sin(weekday*(2.*np.pi/7))
    df1['wkdy_cos'] = np.cos(weekday*(2.*np.pi/7))
    df1['mnth_sin'] = np.sin((mnth-1)*(2.*np.pi/12))
    df1['mnth_cos'] = np.cos((mnth-1)*(2.*np.pi/12))

    df= df1
    df.set_index("timestamp", inplace=True)
    df.ta.strategy('All')
    nan_columns = []
    for column in df.columns:
      if df[column].isnull().sum() > 300:
        nan_columns.append(column)
    df[nan_columns] =df[nan_columns].fillna(0)
    df.reset_index(drop=True, inplace=True)
    return df


  def add_indictors(self,df , moving_a = [7, 9,14,20, 25,50,100, 300], rsi_r = [2,3, 4, 6,10,14,20, 30,100]):
    df[['open', 'high', 'low', 'close', 'volume']] =df[['open', 'high', 'low', 'close', 'volume']].astype('float')
    for range in moving_a:
      df.ta.sma(length=range, append = True)
      df.ta.ema(length=range, append = True)
    for range in rsi_r:
      df.ta.rsi(length=range, append = True)
    df = df.assign(hig_close=df['high'].astype(float) - df['close'].astype(float))
    df = df.assign(low_close=df['low'].astype(float) - df['close'].astype(float))
    df = df.assign(EMA_20_EMA_50=df['EMA_20'].astype(float) - df['EMA_50'].astype(float)) # EMA20 - EMA50 

    #EMA
    df = df.assign(EMA_9_close=df['EMA_7'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(EMA_9_close=df['EMA_9'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(EMA_25_close=df['EMA_25'].astype(float) - df['close'].astype(float)) # EMA25 - close 
    df = df.assign(EMA_100_close=df['EMA_100'].astype(float) - df['close'].astype(float)) # EMA100 - clpse 
    df = df.assign(EMA_20_EMA_50=df['EMA_20'].astype(float) - df['EMA_50'].astype(float)) # EMA20 - EMA50
    df = df.assign(EMA_9_EMA_29=df['EMA_9'].astype(float) - df['EMA_9'].astype(float)) # EMA20 - EMA50

    #SMA
    df = df.assign(SMA_9_close=df['SMA_7'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(SMA_9_close=df['SMA_9'].astype(float) - df['close'].astype(float)) # EMA9- close
    df = df.assign(SMA_25_close=df['SMA_25'].astype(float) - df['close'].astype(float)) # EMA25 - close 
    df = df.assign(SMA_100_close=df['SMA_100'].astype(float) - df['close'].astype(float)) # EMA100 - clpse 
    df = df.assign(SMA_20_EMA_50=df['SMA_20'].astype(float) - df['SMA_50'].astype(float)) # EMA20 - EMA50
    df = df.assign(SMA_9_EMA_29=df['SMA_9'].astype(float) - df['SMA_9'].astype(float)) # EMA20 - EMA50

    try:
      df = df.assign(Taker_seller_base_asset_volume=df['volume'].astype(float) - df['Taker buy base asset volume'].astype(float))
      df = df.assign(Taker_seller_quote_asset_volume=df['Quote asset volume'].astype(float) - df['Taker buy quote asset volume'].astype(float))
    except:pass

    return df

  def get_indictors(self):
    self.df == self.add_time(self.df)
    print("adding indictors...", len(self.df.columns))
    df = add_all_ta_features(self.df,  open="open", high="high", low="low", close="close", volume="volume", fillna=True)
    print("adding ...", len(df.columns))
    df = self.add_indictors(df)
    return df



def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
def prepare_data(df):
  data = df[df.columns]
  data = data.astype(float)
  values_after_scale = scaler(data)
  data= pd.DataFrame(values_after_scale,columns=data.columns)
  return data

In [ ]:
import datetime
datetime.datetime(year= 2016, month= 1, day= 1) #+ datetime.timedelta(minutes=15)
df = df_local[['open', 'high', 'low', 'close', 'volume', 'action']]
dates = pd.date_range(start='2019/1/1' , end = '2022/1/1', periods=len(df))
df['timestamp'] = dates

#df.head(2)
df = df[-1000:]

df = AddIndictors_test(df).get_indictors()
df.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
131it [00:04, 30.09it/s]


adding indictors... 292


/usr/local/lib/python3.7/dist-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


adding ... 378


,open,high,low,close,volume,action,hr_sin,hr_cos,day_sin,day_cos,...,EMA_20_EMA_50,EMA_9_close,EMA_25_close,EMA_100_close,EMA_9_EMA_29,SMA_9_close,SMA_25_close,SMA_100_close,SMA_20_EMA_50,SMA_9_EMA_29
0,48546.44,48557.0,48295.51,48362.41,295.98854,1.0,0.793353,-0.608761,-0.207912,-0.978148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48362.41,48567.1,48359.42,48437.17,337.31191,0.0,0.793353,-0.608761,-0.207912,-0.978148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get all candle patterns (This is the default behaviour)
!pip install TA-Lib
datetime.datetime(year= 2016, month= 1, day= 1) #+ datetime.timedelta(minutes=15)
df = df_local[['open', 'high', 'low', 'close', 'volume', 'action']]
dates = pd.date_range(start='2019/1/1' , end = '2022/1/1', periods=len(df))
df['timestamp'] = dates

df = df.ta.cdl_pattern(name="all")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


AttributeError: ignored

In [ ]:
#df = addIndictors(df_local).get_indictors()
#data = prepare_data(df)
#label = [np.argmax(x) for x in model.predict(data[-1:])][0]
#df = df_local[['open', 'high', ' low', 'close', 'volume', 'action']]
df=x
rsi_r = [2]
ema_list = [7, 9,14,20, 25,50,100, 300]

# df = df.assign(hig_close=df['high'].astype(float) - df['close'].astype(float))
# df = df.assign(low_close=df[' low'].astype(float) - df['close'].astype(float))
for value in rsi_r:
   df.ta.rsi(length=value, append = True)
# for value in ema_list:
#   df.ta.ema(length=value, append = True)



df = clean_dataset(df)
y= df['action']
x = df.drop('action', axis = 1)
#data = prepare_data(x)
#per_importance(x,y,forest)
x.head(2)

In [ ]:
x = prepare_data(x)
forest = features_importance(x,y)

In [ ]:
x.columns

In [ ]:
gb = features_importance_gb(x,y)
anova_feature()

In [ ]:
per_importance(x,y,forest)
per_importance(x,y,gb)

In [ ]:
forest.score(x,y), gb.score(x,y)

In [ ]:
'3outside' in df.columns

In [ ]:
from ta import add_all_ta_features

In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3911    0  3911    0     0  14647      0 --:--:-- --:--:-- --:--:-- 14703
100  503k  100  503k    0     0   560k      0 --:--:-- --:--:-- --:--:--  560k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3911    0  3911    0     0   9378      0 --:--:-- --:--:-- --:--:--  9378
100  406k  100  406k    0     0   397k      0  0:00:01  0:00:01 --:--:-- 3386k


In [ ]:
!pip install TA-Lib

datetime.datetime(year= 2016, month= 1, day= 1) #+ datetime.timedelta(minutes=15)
df = df_local[['open', 'high', 'low', 'close', 'volume', 'action']]
dates = pd.date_range(start='2019/1/1' , end = '2022/1/1', periods=len(df))
df['timestamp'] = dates

df.set_index("timestamp", inplace=True)
df.ta.strategy('All')
nan_columns = []
for column in df.columns:
  if df[column].isnull().sum() > 300:
    nan_columns.append(column)
df[nan_columns] =df[nan_columns].fillna(0)
df.reset_index(drop=True, inplace=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas_ta/core.py:387: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ind_name] = result
/usr/local/lib/python3.7/dist-packages/pandas_ta/core.py:382: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  T

In [ ]:
for c in df.columns:
  print(c)

open
high
low
close
volume
action
AD
ADOSC_3_10
ABER_ZG_5_15
ABER_SG_5_15
ABER_XG_5_15
ABER_ATR_5_15
ACCBL_20
ACCBM_20
ACCBU_20
AMATe_LR_2
AMATe_SR_2
AO_5_34
OBV
OBV_min_2
OBV_max_2
OBVe_4
OBVe_12
AOBV_LR_2
AOBV_SR_2
APO_12_26
AROOND_14
AROONU_14
AROONOSC_14
ATRr_14
BBL_5_2.0
BBM_5_2.0
BBU_5_2.0
BBB_5_2.0
BIAS_SMA_26
BOP
AR_26
BR_26
ADX_14
DMP_14
DMN_14
ALMA_10_6.0_0.85
CCI_14_0.015
CDL_DOJI_10_0.1
CG_10
CHOP_14_1_100
CKSPl_10_1_9
CKSPs_10_1_9
CMF_20
CMO_14
COPC_11_14_10
LDECAY_5
DEC_1
DEMA_10
DCL_20_20
DCM_20_20
DCU_20_20
DPO_20
EBSW_40_10
EFI_13
EMA_10
ENTP_10
EOM_14_100000000
ER_10
BULLP_13
BEARP_13
FISHERT_9_1
FISHERTs_9_1
FWMA_10
HA_open
HA_high
HA_low
HA_close
HILO_13_21
HILOl_13_21
HILOs_13_21
HL2
HLC3
HMA_10
HW-MID
HW-UPPER
HW-LOWER
HWMA_0.2_0.1_0.1
ISA_9
ISB_26
ITS_9
IKS_26
ICS_26
CDL_INSIDE
CFO_9
KAMA_10_2_30
KCLe_20_2
KCBe_20_2
KCUe_20_2
K_9_3
D_9_3
J_9_3
KST_10_15_20_30_10_10_10_15
KSTs_9
INC_1
INERTIA_20_14
LOGRET_1
MACD_12_26_9
MACDh_12_26_9
MACDs_12_26_9
MAD_30
MASSI_9_2

In [ ]:
len(df.columns)

197

In [ ]:
df = AddIndictors_test(df).get_indictors()
# data = prepare_data(df)
# label = [np.argmax(x) for x in model.predict(data[-1:])][0]
# df = df_local[['open', 'high', ' low', 'close', 'volume', 'action']]

df

In [ ]:
import talib
# loading libraries
import pandas as pd
import talib
from talib import abstract

talib.get_function_groups().keys()

dict_keys(['Cycle Indicators', 'Math Operators', 'Math Transform', 'Momentum Indicators', 'Overlap Studies', 'Pattern Recognition', 'Price Transform', 'Statistic Functions', 'Volatility Indicators', 'Volume Indicators'])

In [ ]:
len(df.columns)

285

In [ ]:
df_indicator = pd.DataFrame()
for indicator in talib.get_function_groups()['Cycle Indicators']:
    #df_indicator.append(getattr(abstract, indicator)(df))
    df_indicator = df_indicator.append(getattr(abstract, indicator)(df), ignore_index=True)
    #getattr(abstract, indicator)(df)

In [ ]:
df_indicator

In [ ]:
# list of all indicators
talib.get_function_groups()

In [ ]:
# all Pattern Recognition indicators start with CDL
all_indicators = [method for method in dir(abstract) if method.startswith('CDL')]

In [ ]:
help(abstract)

In [ ]:
df_indicator = pd.DataFrame()
for indicator in all_indicators:
    df[str(indicator)] = getattr(abstract, indicator)(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
